In [1]:
import os
import sys
import numpy as np
import pandas as pd
import igraph as ig
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../python')

from mpl_settings_v3 import *
from plot_auxiliary import *
from planar import create_proximity_graph
from auxiliary import get_base_network_name
from auxiliary import getLinearReg, get_base_network_name, powerlaw, load_delta_data, read_data_file
from plot_auxiliary import load_dataframe, get_critical_measures

%matplotlib inline

net_type = 'DT'
param = 'param'

net_dir = os.path.join('..', 'networks', net_type)

marker_dict = {'meanS': 'o', 'Nsec': 's', 'varSgcc': '^'}
color_dict = {'meanS': 'C0', 'Nsec': 'C1', 'varSgcc': 'C2'}
linestyle_dict = {'meanS': '--', 'Nsec': '-.', 'varSgcc': ':'}
measures_dict = {'meanS': r'$\langle s \rangle$', 'Nsec': r'$S_2 N$', 'varSgcc': r'$\chi$'}
line_colors = {'meanS': 'k', 'Nsec': 'grey', 'varSgcc': 'brown'}

fig_dir = '../draft/videos'

def powerlaw(x, a, c):
    return c*x**a

In [ ]:
def get_label_dict(g):

    components = g.components(mode='WEAK')
    gcc = components.giant()
    membership = np.array(components.membership)
    number, count = np.unique(membership, return_counts=True)
    number = number[np.argsort(count)[::-1]] 
    label_dict = {i: n for i, n in enumerate(number)}
    reverse_label_dict = {n: i for i, n in enumerate(number)}
    
    return label_dict, reverse_label_dict, membership
    
L = 256
size = L*L
attack = 'Btw'
network = 'DT'
seed = 0

marker_sizes = {
    32: {
        'backbone': 80,
        'removed': 60,
        'components': 40
    },
    64: {
        'backbone': 30,
        'removed': 20,
        'components': 15
    },
    128: {
        'backbone': 20,
        'removed': 10,
        'components': 6
    },
    256: {
        'backbone': 5,
        'removed': 3,
        'components': 1
    },
    512: {
        'backbone': 3,
        'removed': 1,
        'components': 0.1
    },
}

s0 = marker_sizes[L]['backbone']
s1 = marker_sizes[L]['removed']
s2 = marker_sizes[L]['components']

backbone = True
components = False

if backbone and components:
    info = 'all'
elif backbone:
    info = 'backbone'
elif components:
    info = 'components'

for ii, a in enumerate(np.arange(0, 1.01, 0.01)):
    print(a)
    ## Spanning subgraph of the removed nodes
    h, max_pos, _ = load_delta_data(network, size, param, attack, seed)
    th = int(a*max_pos)+1

    attack_order = h['attack_order']
    to_delete = set(h.vs['oi']).difference(set(attack_order[:th]))
    h.delete_vertices(to_delete)

    backbone_label_dict, _, backbone_membership = get_label_dict(h)

    backbone_index_dict = {oi: i for i, oi in enumerate(h.vs['oi'])}

    ################

    _colors = [f'C{i}' for i in range(1000)] #+ 10000*['white']
    g, _, _ = load_delta_data(network, size, param, attack, seed)    
    position = np.array(g.vs['position'])
    attack_order = g['attack_order']
    to_delete = set(g.vs['oi']).difference(set(attack_order[th:]))
    g.delete_vertices(to_delete)

    label_dict, reverse_label_dict, membership = get_label_dict(g)

    index_dict = {oi: i for i, oi in enumerate(g.vs['oi'])}

    if False:
        attack_order = g['attack_order']
        g.delete_vertices(attack_order[:th])

        components = g.components(mode='WEAK')
        gcc = components.giant()
        membership = np.array(components.membership)
        number, count = np.unique(membership, return_counts=True)
        number = number[np.argsort(count)[::-1]] 
        count = np.sort(count)[::-1]

        label_dict = dict((i, n) for i,n in enumerate(number))
        for idx in np.where(count==1)[0]:
            label_dict[idx] = -1

    #print('start plot')

    def plot_backbone():
        _position = []
        for oi in attack_order[:th]:
            #idx = h.vs['oi'].index(oi)
            idx = backbone_index_dict[oi]
            #if backbone_label_dict[backbone_membership[idx]] == 0:
            if backbone_membership[idx] == backbone_label_dict[0]:
                _position.append(position[oi])
        _position = np.array(_position)
        if _position.shape[0] != 0:
            X, Y = _position[:,0], _position[:,1]
            plt.scatter(X, Y, c='k', marker='*', s=s0, zorder=10)

    def plot_removed():
        _position = []
        for oi in attack_order[:th]:
            #idx = h.vs['oi'].index(oi)
            idx = backbone_index_dict[oi]
            if backbone_membership[idx] != backbone_label_dict[0]:
                _position.append(position[oi])
        _position = np.array(_position)
        if _position.shape[0] != 0:
            X, Y = _position[:,0], _position[:,1]
            plt.scatter(X, Y, c='0.4', marker='s', s=s1, zorder=5)

    def plot_components():
        _position = position[attack_order[th:]]
        X, Y = _position[:,0], _position[:,1]
        c_values = []

        for oi in attack_order[th:]:
            idx = index_dict[oi]
            c_values.append(
                #_colors[reverse_label_dict[membership[idx]]]
                'C{}'.format(reverse_label_dict[membership[idx]]%100)
            )
        #print('start scatter', flush=True)
        plt.scatter(
            X, Y, c=c_values, marker='.', s=s2, zorder=0
        )

    def save_fig(info, frame_idx):
        fig_name = os.path.join(
            fig_dir, 
            '{}_{}_L{}_{}_{:05d}_frame{:05d}'.format(info, net_type, L, attack, seed, frame_idx)
        )
        dpi = 100
        plt.savefig(fig_name + '.png', transparent=False, dpi=dpi)

    fig, ax = plt.subplots(figsize=(10,10))
    #mpl.style.use('default')
    #ax.set_xlabel(r'$x$')
    #ax.set_ylabel(r'$y$')
    #ax.set_xlim(0, L)
    #ax.set_ylim(0, L)
    ax.set_xticks([])
    ax.set_yticks([])
    #ax.margins()
    ax.set_facecolor('white')
    sns.despine(bottom=True, left=True)
    if backbone:
        #print('Plot backbone')
        plot_backbone()
        #print('Plot removed')
        plot_removed()
    if components:
        #print('Plot components')
        plot_components()
    save_fig(info, ii)

    #plt.show()

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14


In [4]:
backbone_index_dict

{3824: 0}